**CAPSTONE EASY MONEY // 02SDCESP0223 - GRUPO 7**

## Introducción al Capstone Fintech -Easy Money

Este proyecto tiene como objetivo realizar un caso semejante a la vida real, en el que realizaremos las tareas solicitadas a Bob, un nuevo trabajador de una compañía comercializadora de productos financieros llamada Easy Money.

A Bob se le ha contratado debido a sus conocimientos en Data Science, debido a que la empresa requiere incorporar la analítica en todos los procesos core. La empresa cuenta con un autoservicio de BI con ReportServer por lo que podemos considerar que se encuentra en la fase 4. Reporting Avanzado / estratégico del Módelo de Madurez de Analytics, por lo que quedaría pendiente trabajar en la modelización predictiva de los productos a vender y la analítica prescriptiva de la empresa.

Es por ello, que empezamos conociendo este Notebook, la información de la empresa proporcionada por el área de IT, la cual consta de 5 archivos .CSV que resumiremos a continuación y en el que pueden conocer más en el archivo adjunto Tablas - Descripción.xlsx.

Se nos han entregado 5 archivos:
1. **customer_commercial_Activity**: con 5.962.924 records (filas), donde se otorga una primera vista a la base de clientes de Easy Money, la cual puede entenderse como la base de información de cada cliente durante los meses analizados (pk_partition)

2. **customer_products**: con 5.962.924 records (filas), donde se observa los productos activos de cada cliente durante los meses analizados (pk_partition). En su mayoría son columnas booleanas (para los productos) y permite observar la evolución (altas y bajas) de los productos que cada cliente mantiene con Easy Money.

3. **customer_sociodemographic**: con 5.962.924 records (filas), se observa la información personal de cada cliente durante los meses analizados (pk_partition). Esta información nos permite conocer más a los clientes y segmentar de la mejor manera.

4. **sales**: con 240.772 records (filas), se observar las ventas realizadas durante 16 meses (month_Sale). Además, el producto y el margen neto cada venta.

5. **product_description**: con 13 records (filas), se hace referencia a los productos y familia de productos de Easy money.

Para el correcto preprocesamiento de las tablas, hemos realizado una primera vista buscando duplicados y nulos. Posteriormente, se ha trabajado cada columna para homogenizar las variables, trabajar los nulos y cambiar el tipo de variable.

Posteriormente, hemos creado el modelo relacional de los datos. Donde en una primera instancia se ha realizado join de las tablas customer_commercial_Activity, customer_products y customer_sociodemographic, para trabajar la información de cada cliente en una misma tabla. Por otro lado, se ha realizado un join para las tablas de sales y product_description. De esta manera, conoceremos el productos y su familia para cada venta realizada.

Realizado este procesamiento y teniendo una mejor visión de la información, hemos creado nuevas variables y nuevas tablas que nos ayudarán a profundizar en el entendimeinto del negocio.

Al final del proyecto, tendremos un entendimiento más profundo de la información de los clientes y las ventas realizadas.

Trabajo realizado por:
* **Francisco Camacho**
* **Mariela Drago**

### Montamos drive e importamos las librerías necesarias

In [ ]:
# Montamos Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

plt.style.use("ggplot")

from google.colab import files

from sklearn import model_selection
import xgboost as xgb

from scipy import stats
from datetime import datetime
import random

#import xgboost as xgb
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier

### Cargamos los datos facilitados

In [ ]:
# Cargar los datos

DATA_PATH = "/content/drive/MyDrive/MDS - Grupo 7/Capstone/datasets_TFM + diccionario"
customer_commercial_activity = pd.read_csv(DATA_PATH+ "/customer_commercial_activity.csv")
customer_products = pd.read_csv(DATA_PATH + "/customer_products.csv")
customer_sociodemographics = pd.read_csv(DATA_PATH + "/customer_sociodemographics.csv")
sales = pd.read_csv(DATA_PATH + "/sales.csv")
product_description = pd.read_csv(DATA_PATH + "/product_description.csv")

In [ ]:
customer_commercial_activity.name = 'customer_commercial_activity'
customer_products.name = 'customer_products'
customer_sociodemographics.name = 'customer_sociodemographics'
sales.name = 'sales'
product_description.name = 'product_description'

In [ ]:
df_juntos = {'customer_commercial_activity': customer_commercial_activity, 'customer_products': customer_products, 'customer_sociodemographics': customer_sociodemographics, 'sales': sales, 'product_description':product_description}


# **1. Data Understanding y Data cleaning**

In [ ]:
#Descargamos el info y describe de cada tabla para conocer a detalle cada variable

In [ ]:
def download_csv(df):
  download1 = df.describe(include = 'all').T
  download1.to_csv(f'{df.name}_describe.csv')
  return files.download (f'{df.name}_describe.csv')

In [ ]:
#download_csv(customer_commercial_activity)
#download_csv(customer_products)
#download_csv(customer_sociodemographics)
#download_csv(sales)
#download_csv(product_description)

In [ ]:
def info_data (*dataframes):
  for df in dataframes:
    print(f"Información del dataframe {df.name}:")
    df.info()
    print("\n")


In [ ]:
info_data (customer_commercial_activity, customer_products, customer_sociodemographics, sales, product_description)


Información del dataframe customer_commercial_activity:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   pk_cid           int64  
 2   pk_partition     object 
 3   entry_date       object 
 4   entry_channel    object 
 5   active_customer  float64
 6   segment          object 
dtypes: float64(1), int64(2), object(4)
memory usage: 318.5+ MB


Información del dataframe customer_products:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   pk_cid              int64  
 2   pk_partition        object 
 3   short_term_deposit  int64  
 4   loans               int64  
 5   mortgage            int64  
 6   funds               int64  
 7   securities          int64  
 8   lon

In [ ]:
customer_commercial_activity[(customer_commercial_activity["pk_cid"]==1387215)]

,Unnamed: 0,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
584084,1667901,1387215,2018-03,2018-03,KHK,1.0,01 - TOP
836838,2305122,1387215,2018-04,2018-03,KHK,1.0,01 - TOP
1121946,2727756,1387215,2018-05,2018-03,KHK,1.0,01 - TOP
1371157,3357237,1387215,2018-06,2018-03,KHK,1.0,01 - TOP
1771509,4561710,1387215,2018-07,2018-03,KHK,1.0,01 - TOP
1990769,5077214,1387215,2018-08,2018-03,KHK,1.0,01 - TOP
2453837,6224434,1387215,2018-09,2018-03,KHK,1.0,01 - TOP
2772483,6590990,1387215,2018-10,2018-03,KHK,1.0,01 - TOP
3190683,7820090,1387215,2018-11,2018-03,KHK,1.0,01 - TOP
3773815,9013628,1387215,2018-12,2018-03,KHK,1.0,01 - TOP


In [ ]:
def describe_num(dataframes):
    df_descriptions = pd.DataFrame()

    for name, df in dataframes.items():
        description = df.describe(include = np.number).T
        description['DataFrame'] = name
        df_descriptions = pd.concat([df_descriptions, description])

    return df_descriptions


In [ ]:
describe_num(df_juntos)

,count,mean,std,min,25%,50%,75%,max,DataFrame
Unnamed: 0,5962924.0,7.155425e+06,3.920784e+06,0.00,4.240386e+06,7499346.50,10201422.25,13647308.00,customer_commercial_activity
pk_cid,5962924.0,1.234930e+06,1.623020e+05,15891.00,1.112532e+06,1231097.00,1352339.00,1553689.00,customer_commercial_activity
active_customer,5962924.0,4.033845e-01,4.905767e-01,0.00,0.000000e+00,0.00,1.00,1.00,customer_commercial_activity
Unnamed: 0,5962924.0,7.155425e+06,3.920784e+06,0.00,4.240386e+06,7499346.50,10201422.25,13647308.00,customer_products
pk_cid,5962924.0,1.234930e+06,1.623020e+05,15891.00,1.112532e+06,1231097.00,1352339.00,1553689.00,customer_products
short_term_deposit,5962924.0,2.581619e-03,5.074401e-02,0.00,0.000000e+00,0.00,0.00,1.00,customer_products
loans,5962924.0,7.848498e-05,8.858828e-03,0.00,0.000000e+00,0.00,0.00,1.00,customer_products
mortgage,5962924.0,5.433576e-05,7.371080e-03,0.00,0.000000e+00,0.00,0.00,1.00,customer_products
funds,5962924.0,3.370662e-03,5.795948e-02,0.00,0.000000e+00,0.00,0.00,1.00,customer_products
securities,5962924.0,3.711770e-03,6.081113e-02,0.00,0.000000e+00,0.00,0.00,1.00,customer_products


In [ ]:
def describe_cat(dataframes):
    df_descriptions = pd.DataFrame()

    for name, df in dataframes.items():
        description = df.describe(exclude = np.number).T
        description['DataFrame'] = name
        df_descriptions = pd.concat([df_descriptions, description])

    return df_descriptions

In [ ]:
describe_cat(df_juntos)

,count,unique,top,freq,DataFrame
pk_partition,5962924,17,2019-05,442995,customer_commercial_activity
entry_date,5962924,53,2016-10,382473,customer_commercial_activity
entry_channel,5829891,68,KHE,3113947,customer_commercial_activity
segment,5828980,3,03 - UNIVERSITARIO,3900166,customer_commercial_activity
pk_partition,5962924,17,2019-05,442995,customer_products
pk_partition,5962924,17,2019-05,442995,customer_sociodemographics
country_id,5962924,41,ES,5960672,customer_sociodemographics
gender,5962899,2,H,3087502,customer_sociodemographics
deceased,5962924,2,N,5961849,customer_sociodemographics
month_sale,240773,16,2018-10-01,27693,sales


### 1.1 Nulos y duplicados en todos los DataFrames

In [ ]:
def info_nulls(*dataframes):
  for df in dataframes:
    print(f"{df.name} - Valores nulos:")
    print(df.isnull().sum())
    print("\n")


In [ ]:
info_nulls(customer_commercial_activity, customer_products, customer_sociodemographics, sales, product_description)

customer_commercial_activity - Valores nulos:
Unnamed: 0              0
pk_cid                  0
pk_partition            0
entry_date              0
entry_channel      133033
active_customer         0
segment            133944
dtype: int64


customer_products - Valores nulos:
Unnamed: 0             0
pk_cid                 0
pk_partition           0
short_term_deposit     0
loans                  0
mortgage               0
funds                  0
securities             0
long_term_deposit      0
em_account_pp          0
credit_card            0
payroll               61
pension_plan          61
payroll_account        0
emc_account            0
debit_card             0
em_account_p           0
em_acount              0
dtype: int64


customer_sociodemographics - Valores nulos:
Unnamed: 0            0
pk_cid                0
pk_partition          0
country_id            0
region_code        2264
gender               25
age                   0
deceased              0
salary          15411

In [ ]:
def info_duplicate(*dataframes):
  for df in dataframes:
    print(f"{df.name} - Filas duplicadas: {df.duplicated().any()}")
    print("\n")

In [ ]:
info_duplicate(customer_commercial_activity, customer_products, customer_sociodemographics, sales, product_description)

customer_commercial_activity - Filas duplicadas: False


customer_products - Filas duplicadas: False


customer_sociodemographics - Filas duplicadas: False


sales - Filas duplicadas: False


product_description - Filas duplicadas: False




# **2. Data Preparation**

##Creamos una copia de seguridad

In [ ]:
customer_commercial_activity1 = customer_commercial_activity.copy()
customer_products1 = customer_products.copy()
customer_sociodemographics1 = customer_sociodemographics.copy()
sales1 = sales.copy()
product_description1 = product_description.copy()

In [ ]:
customer_commercial_activity1.name = 'customer_commercial_activity1'
customer_products1.name = 'customer_products1'
customer_sociodemographics1.name = 'customer_sociodemographics1'
sales1.name = 'sales1'
product_description1.name = 'product_description1'


##Borramos Unnamed: 0

In [ ]:
def borrar_col(column, *dataframes):
  for df in dataframes:
    if column in df.columns:
      df.drop(column, axis=1, inplace=True)
  return "Se borró la columna"

In [ ]:
borrar_col("Unnamed: 0", customer_commercial_activity1, customer_products1, customer_sociodemographics1, sales1, product_description1)

'Se borró la columna'

In [ ]:
customer_commercial_activity1

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO
4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO
...,...,...,...,...,...,...
5962919,1166765,2019-05,2016-08,KHE,0.0,03 - UNIVERSITARIO
5962920,1166764,2019-05,2016-08,KHE,0.0,03 - UNIVERSITARIO
5962921,1166763,2019-05,2016-08,KHE,1.0,02 - PARTICULARES
5962922,1166789,2019-05,2016-08,KHE,0.0,03 - UNIVERSITARIO


## 2.1 customer_commercial_activity

### Analizamos *customer_commercial_activity*

In [ ]:
def mostrar_informacion_dataframe(df):
  print(f"Información de: {df.name}")
  df.info()

  for columna in df.columns:
        print("\nRecuento de valores únicos en la columna:")
        print(df[columna].value_counts())
        print("\n")
        print("\nRecuento de valores nulos en la columna:")
        print(df[columna].isnull().sum())
        print("\n")

In [ ]:
mostrar_informacion_dataframe(customer_commercial_activity1)

Información de: customer_commercial_activity1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   pk_cid           int64  
 1   pk_partition     object 
 2   entry_date       object 
 3   entry_channel    object 
 4   active_customer  float64
 5   segment          object 
dtypes: float64(1), int64(1), object(4)
memory usage: 273.0+ MB

Recuento de valores únicos en la columna:
1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
1470941     1
1521049     1
1550586     1
Name: pk_cid, Length: 456373, dtype: int64



Recuento de valores nulos en la columna:
0



Recuento de valores únicos en la columna:
2019-05    442995
2019-04    439627
2019-03    436183
2019-02    431727
2019-01    426875
2018-12    422481
2018-11    416387
2018-10    402300
2018-09    375323
2018-08    352922
2018-07    339339
2018-06  

Análisis:


*   pk_cid: cambiar a object. - debemos evaluar porque se tienen hasta 17 datos por pk_cid. revisar si se borra
*   pk_partition: es una fecha, cambiar a datetime64 (%Y, %m)
*   entry_date: es una fecha, cambiar a datetime64 (%Y, %m)
*   entry_channel: tiene valores nulos (133033), completar con (no_info)
*   active_customer: boolean, cambiar a int8 (para menor almacenaje)
*   segment: ok, tiene valores nulos (133944), completar con (no_indica)







#### pk_cid

In [ ]:
customer_commercial_activity1["pk_cid"] = customer_commercial_activity1["pk_cid"].astype(str)


In [ ]:
customer_commercial_activity1[customer_commercial_activity1["pk_cid"] == "1375586"]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
346322,1375586,2018-02,2018-01,KHL,1.0,02 - PARTICULARES
592407,1375586,2018-03,2018-01,KHL,1.0,02 - PARTICULARES
831104,1375586,2018-04,2018-01,KHL,1.0,02 - PARTICULARES
1108457,1375586,2018-05,2018-01,KHL,1.0,02 - PARTICULARES
1357280,1375586,2018-06,2018-01,KHL,1.0,02 - PARTICULARES
1791360,1375586,2018-07,2018-01,KHL,1.0,02 - PARTICULARES
1988936,1375586,2018-08,2018-01,KHL,0.0,02 - PARTICULARES
2487565,1375586,2018-09,2018-01,KHL,0.0,02 - PARTICULARES
2753984,1375586,2018-10,2018-01,KHL,0.0,02 - PARTICULARES


Comentario: El pk_cid, se repite porque hay clientes que se encuentran activos por varios meses, siendo el pk_partition los meses que se encuentra activo

In [ ]:
customer_commercial_activity1["pk_cid"].nunique()

456373

#### pk_partition

In [ ]:
customer_commercial_activity1["pk_partition"] = pd.to_datetime(customer_commercial_activity1["pk_partition"], format= "%Y-%m")

####entry_date

In [ ]:
customer_commercial_activity1["entry_date"] = pd.to_datetime(customer_commercial_activity1["entry_date"], format= "%Y-%m")

####entry_channel

In [ ]:
def valores_nulos(df, columna, relleno):
  df[columna].fillna(relleno, inplace=True)
  return df[columna].isnull().sum()

In [ ]:
valores_nulos(customer_commercial_activity1, "entry_channel", "No_data")

0

####active_customer

In [ ]:
customer_commercial_activity1["active_customer"] = customer_commercial_activity1["active_customer"].astype("int8")

####segment

In [ ]:
valores_nulos(customer_commercial_activity1, "segment", "no_indica")

0

In [ ]:
mostrar_informacion_dataframe(customer_commercial_activity1)

Información de: customer_commercial_activity1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   pk_cid           object        
 1   pk_partition     datetime64[ns]
 2   entry_date       datetime64[ns]
 3   entry_channel    object        
 4   active_customer  int8          
 5   segment          object        
dtypes: datetime64[ns](2), int8(1), object(3)
memory usage: 233.2+ MB

Recuento de valores únicos en la columna:
1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
1470941     1
1521049     1
1550586     1
Name: pk_cid, Length: 456373, dtype: int64



Recuento de valores nulos en la columna:
0



Recuento de valores únicos en la columna:
2019-05-01    442995
2019-04-01    439627
2019-03-01    436183
2019-02-01    431727
2019-01-01    426875
2018-12-01    422481
2018-11-01    41638

## 2.2 customer_products

###Analizamos *customer_products*

In [ ]:
mostrar_informacion_dataframe(customer_products1)

Información de: customer_products1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 17 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   em_account_pp       int64  
 9   credit_card         int64  
 10  payroll             float64
 11  pension_plan        float64
 12  payroll_account     int64  
 13  emc_account         int64  
 14  debit_card          int64  
 15  em_account_p        int64  
 16  em_acount           int64  
dtypes: float64(2), int64(14), object(1)
memory usage: 773.4+ MB

Recuento de valores únicos en la columna:
1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
14709

Análisis:


*   pk_cid: cambiar a object.
*   pk_partition: es una fecha, cambiar a datetime64 (%Y, %m)
*   Hacer una lista con las columnas de productos y cambiarlas a "int8", los nulos se consideran como 0 porque no tenemos información de si cuentan con ese producto o no. (deben ser booleans)

#### pk_cid

In [ ]:
customer_products1["pk_cid"] = customer_products1["pk_cid"].astype(str)


####pk_partition

In [ ]:
customer_products1["pk_partition"] = pd.to_datetime(customer_products1["pk_partition"], format= "%Y-%m")

####Productos

In [ ]:
customer_products1.columns.tolist()

['pk_cid',
 'pk_partition',
 'short_term_deposit',
 'loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount']

In [ ]:
lista_productos = [ 'short_term_deposit',
 'loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount']

In [ ]:
def cambiar_productos (df, columnas):
  df[columnas] = df[columnas].replace([np.nan, np.inf, -np.inf], 0)
  df[columnas] = df[columnas].astype("int8")
  return df.info()

In [ ]:
cambiar_productos (customer_products1, lista_productos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              object        
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  int8          
 3   loans               int8          
 4   mortgage            int8          
 5   funds               int8          
 6   securities          int8          
 7   long_term_deposit   int8          
 8   em_account_pp       int8          
 9   credit_card         int8          
 10  payroll             int8          
 11  pension_plan        int8          
 12  payroll_account     int8          
 13  emc_account         int8          
 14  debit_card          int8          
 15  em_account_p        int8          
 16  em_acount           int8          
dtypes: datetime64[ns](1), int8(15), object(1)
memory usage: 176.3+ MB


In [ ]:
mostrar_informacion_dataframe(customer_products1)

Información de: customer_products1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              object        
 1   pk_partition        datetime64[ns]
 2   short_term_deposit  int8          
 3   loans               int8          
 4   mortgage            int8          
 5   funds               int8          
 6   securities          int8          
 7   long_term_deposit   int8          
 8   em_account_pp       int8          
 9   credit_card         int8          
 10  payroll             int8          
 11  pension_plan        int8          
 12  payroll_account     int8          
 13  emc_account         int8          
 14  debit_card          int8          
 15  em_account_p        int8          
 16  em_acount           int8          
dtypes: datetime64[ns](1), int8(15), object(1)
memory usage: 176.3+ MB

Recuento de valores 

In [ ]:
#vamos a esperar para eliminar "em_account_pp" que no tiene información

In [ ]:
stock_mes = customer_products1.groupby('pk_partition').sum(numeric_only=True)
stock_mes

,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
pk_partition,,,,,,,,,,,,,,,
2018-01-01,883,19,15,786,932,4884,0,3325,8145,8835,13478,15320,24696,2,215293
2018-02-01,1343,19,15,872,965,4956,0,3282,8889,9496,14246,15827,25448,2,217098
2018-03-01,1664,23,17,969,965,4965,0,3511,9735,9882,14989,16428,27046,2,218683
2018-04-01,1740,24,17,1084,983,5107,0,3725,9905,10560,15825,16947,27764,2,219816
2018-05-01,1455,27,18,1145,1012,5356,0,3797,9994,10148,16697,17569,27911,2,221291
2018-06-01,1244,27,19,1159,1019,5588,0,4002,10946,11635,15540,17950,28828,2,224328
2018-07-01,1282,28,20,1172,1053,5796,0,4137,11888,12779,16916,18185,29578,2,234324
2018-08-01,1335,29,20,1219,1133,5923,0,4206,11383,12126,18118,18333,29205,2,245980
2018-09-01,1386,31,20,1237,1201,6158,0,4289,11850,12624,18862,18618,31684,2,261210


## 2.3 customer_sociodemographics

### Analizamos customer_sociodemographics

In [ ]:
mostrar_informacion_dataframe(customer_sociodemographics1)

Información de: customer_sociodemographics1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   pk_cid        int64  
 1   pk_partition  object 
 2   country_id    object 
 3   region_code   float64
 4   gender        object 
 5   age           int64  
 6   deceased      object 
 7   salary        float64
dtypes: float64(2), int64(2), object(4)
memory usage: 363.9+ MB

Recuento de valores únicos en la columna:
1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
1470941     1
1521049     1
1550586     1
Name: pk_cid, Length: 456373, dtype: int64



Recuento de valores nulos en la columna:
0



Recuento de valores únicos en la columna:
2019-05    442995
2019-04    439627
2019-03    436183
2019-02    431727
2019-01    426875
2018-12    422481
2018-11    416387
2018-10    402300
2018-09    375323
2018-08    352922

Análisis:


*   pk_cid: cambiar a object.
*   pk_partition: es una fecha, cambiar a datetime64 (%Y, %m)
*   country_id: ok
*   region_code: tiene nulos, rellenar con "no_data"
*   gender: tiene nulos, rellenar con "no_data"
*   age: cambiar a int8
*   decease: boolean, ok
*   salary: cambiar a "int"



#### pk_cid

In [ ]:
customer_sociodemographics1["pk_cid"] = customer_sociodemographics1["pk_cid"].astype(str)


####pk_partition

In [ ]:
customer_sociodemographics1["pk_partition"] = pd.to_datetime(customer_sociodemographics1["pk_partition"], format= "%Y-%m")

####region_code

In [ ]:
valores_nulos(customer_sociodemographics1, "region_code", 0)

0

In [ ]:
customer_sociodemographics1["region_code"] = customer_sociodemographics1["region_code"].astype(str)

####gender

In [ ]:
valores_nulos(customer_sociodemographics1, "gender", "no_data")

0

In [ ]:
customer_sociodemographics1["gender"] = customer_sociodemographics1["gender"].replace("H", "M")
customer_sociodemographics1["gender"] = customer_sociodemographics1["gender"].replace("V", "H")

####age

In [ ]:
customer_sociodemographics1["age"] = customer_sociodemographics1["age"].astype("int8")

####salary

In [ ]:
customer_sociodemographics1["salary"] = customer_sociodemographics1["salary"].round()

In [ ]:
# Para tratar los nulos de Salary, primero conoceremos más de la variable. R

salary_no_null = customer_sociodemographics1.dropna(subset=['salary'])

salary_by_region = salary_no_null.groupby('region_code')['salary'].agg(
    mean_salary= 'mean',
    max_salary= 'max',
    min_salary= 'min',
    median_salary= 'median'
)
salary_by_region.T


region_code,0.0,1.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,...,48.0,49.0,5.0,50.0,51.0,52.0,6.0,7.0,8.0,9.0
mean_salary,175569.405405,107098.0,7.351092e+04,9.468972e+04,76509.490885,67375.639062,7.976911e+04,1.078846e+05,70310.261291,1.458372e+05,...,110240.885714,8.148438e+04,7.727128e+04,1.080285e+05,2.109242e+05,1.649124e+05,69921.475043,1.531694e+05,1.568012e+05,9.545358e+04
max_salary,385514.000000,253563.0,1.309035e+06,2.311469e+06,668527.000000,952513.000000,1.394465e+06,2.564976e+06,408454.000000,6.209401e+06,...,160481.000000,1.536265e+06,2.768593e+06,8.516913e+06,4.082464e+06,1.949948e+06,594804.000000,1.571172e+07,5.752268e+06,1.446904e+06
min_salary,46884.000000,55271.0,5.130000e+03,8.346000e+03,11933.000000,3732.000000,7.507000e+03,9.781000e+03,8843.000000,1.868100e+04,...,55588.000000,7.776000e+03,7.290000e+03,9.788000e+03,3.343000e+04,3.443700e+04,7144.000000,6.698000e+03,7.983000e+03,1.115400e+04
median_salary,114949.000000,74454.0,6.640600e+04,7.551400e+04,64852.000000,60829.000000,6.437100e+04,9.405400e+04,67615.000000,1.082300e+05,...,108659.000000,7.341700e+04,6.760900e+04,9.679600e+04,1.241580e+05,1.136100e+05,60290.000000,1.167100e+05,1.246220e+05,8.768100e+04


In [ ]:
#Utilizamos 'region_code', ya que la mayoría es de España y decidimos ponerle 0.0 a los nulos

In [ ]:
salary_no_null[salary_no_null['region_code']=="0.0"]

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
2193581,1250880,2018-09-01,RO,0.0,H,52,N,114949.0
2875968,1250880,2018-10-01,RO,0.0,H,52,N,114949.0
2976453,1440250,2018-11-01,DZ,0.0,H,39,N,385514.0
3089135,1250880,2018-11-01,RO,0.0,H,52,N,114949.0
3447353,1250880,2018-12-01,RO,0.0,H,52,N,114949.0
3628803,670953,2018-12-01,GB,0.0,H,39,N,111760.0
3650220,1440250,2018-12-01,DZ,0.0,H,39,N,385514.0
3793909,1440250,2019-01-01,DZ,0.0,H,39,N,385514.0
4067864,1250880,2019-01-01,RO,0.0,H,52,N,114949.0
4103151,670953,2019-01-01,GB,0.0,H,40,N,111760.0


In [ ]:
median_by_region = salary_by_region['median_salary']
customer_sociodemographics1['salary'] = customer_sociodemographics1.apply(lambda row: median_by_region[row['region_code']] if pd.isnull(row['salary']) else row['salary'], axis=1)


In [ ]:
customer_sociodemographics1.head(20)

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01-01,ES,29.0,M,35,N,87218.0
1,1050611,2018-01-01,ES,13.0,H,23,N,35549.0
2,1050612,2018-01-01,ES,13.0,H,23,N,122179.0
3,1050613,2018-01-01,ES,50.0,M,22,N,119776.0
4,1050614,2018-01-01,ES,50.0,H,23,N,96796.0
5,1050615,2018-01-01,ES,45.0,M,23,N,22220.0
6,1050616,2018-01-01,ES,24.0,M,23,N,295590.0
7,1050617,2018-01-01,ES,50.0,M,23,N,113317.0
8,1050619,2018-01-01,ES,20.0,M,24,N,53029.0
9,1050620,2018-01-01,ES,10.0,M,23,N,113195.0


In [ ]:
mostrar_informacion_dataframe(customer_sociodemographics1)

Información de: customer_sociodemographics1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   pk_cid        object        
 1   pk_partition  datetime64[ns]
 2   country_id    object        
 3   region_code   object        
 4   gender        object        
 5   age           int8          
 6   deceased      object        
 7   salary        float64       
dtypes: datetime64[ns](1), float64(1), int8(1), object(5)
memory usage: 324.1+ MB

Recuento de valores únicos en la columna:
1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
1470941     1
1521049     1
1550586     1
Name: pk_cid, Length: 456373, dtype: int64



Recuento de valores nulos en la columna:
0



Recuento de valores únicos en la columna:
2019-05-01    442995
2019-04-01    439627
2019-03-01    436183
2019-02-01    431727
2019-01

## 2.4 sales

### Analizamos *sales*

In [ ]:
mostrar_informacion_dataframe(sales1)

Información de: sales1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240773 entries, 0 to 240772
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   pk_sale     240773 non-null  int64  
 1   cid         240773 non-null  int64  
 2   month_sale  240773 non-null  object 
 3   product_ID  240773 non-null  int64  
 4   net_margin  240773 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 9.2+ MB

Recuento de valores únicos en la columna:
6666      1
167186    1
167174    1
167175    1
167176    1
         ..
86928     1
86929     1
86930     1
86931     1
247438    1
Name: pk_sale, Length: 240773, dtype: int64



Recuento de valores nulos en la columna:
0



Recuento de valores únicos en la columna:
1387215    10
1436512     9
1380507     9
1402957     9
1402199     9
           ..
1409414     1
1409413     1
1409412     1
1409410     1
1553571     1
Name: cid, Length: 152754, dtype: int64



Recu

Análisis:


*   pk_sale:ok, 100% valores únicos
*   cid: cambiar a object
*   month_sale: cambiar a datetime
*   product_ID: cambiar a object
*   net_margin: ok






#### cid

In [ ]:
sales1["cid"] = sales1["cid"].astype(str)

####month_sale

In [ ]:
sales1["month_sale"] = pd.to_datetime(sales1["month_sale"], format= "%Y-%m-%d")

####product_ID

In [ ]:
sales1["product_ID"] = sales1["product_ID"].astype(str)

In [ ]:
mostrar_informacion_dataframe(sales1)

Información de: sales1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240773 entries, 0 to 240772
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   pk_sale     240773 non-null  int64         
 1   cid         240773 non-null  object        
 2   month_sale  240773 non-null  datetime64[ns]
 3   product_ID  240773 non-null  object        
 4   net_margin  240773 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 9.2+ MB

Recuento de valores únicos en la columna:
6666      1
167186    1
167174    1
167175    1
167176    1
         ..
86928     1
86929     1
86930     1
86931     1
247438    1
Name: pk_sale, Length: 240773, dtype: int64



Recuento de valores nulos en la columna:
0



Recuento de valores únicos en la columna:
1387215    10
1436512     9
1380507     9
1402957     9
1402199     9
           ..
1409414     1
1409413     1
1409412     1
1409410

## 2.5 product_description

### 1.6.1 Analizamos *product_description*

In [ ]:
mostrar_informacion_dataframe(product_description1)

Información de: product_description1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pk_product_ID   13 non-null     int64 
 1   product_desc    13 non-null     object
 2   family_product  13 non-null     object
dtypes: int64(1), object(2)
memory usage: 440.0+ bytes

Recuento de valores únicos en la columna:
4657    1
3819    1
1364    1
2234    1
2235    1
8871    1
2312    1
2335    1
2336    1
2673    1
1119    1
9001    1
4033    1
Name: pk_product_ID, dtype: int64



Recuento de valores nulos en la columna:
0



Recuento de valores únicos en la columna:
em_acount             1
debit_card            1
pension_plan          1
payroll               1
payroll_account       1
emc_account           1
credit_card           1
short_term_deposit    1
long_term_deposit     1
securities            1
funds                 1
loans                 1


Analizamos:

*   pk_product_ID: cambiar a object



In [ ]:
product_description1["pk_product_ID"] = product_description1["pk_product_ID"].astype(str)

In [ ]:
product_description1

,pk_product_ID,product_desc,family_product
0,4657,em_acount,account
1,3819,debit_card,payment_card
2,1364,pension_plan,pension_plan
3,2234,payroll,account
4,2235,payroll_account,account
5,8871,emc_account,account
6,2312,credit_card,payment_card
7,2335,short_term_deposit,investment
8,2336,long_term_deposit,investment
9,2673,securities,investment


##Relación pk_cid y pk_partition

In [ ]:
#def filtrar_dataframe(df):
#    df_filtrado = df.groupby('pk_cid').agg({'pk_partition': 'max'}).reset_index()
#    df_filtrado = df_filtrado.merge(df, on=['pk_cid', 'pk_partition'], how='left')
#    return df_filtrado

In [ ]:
# df_filtrado = filtrar_dataframe(customer_commercial_activity1)

In [ ]:
#df_filtrado

In [ ]:
#df_filtrado.name = "df_filtrado"

In [ ]:
#mostrar_informacion_dataframe(df_filtrado)

In [ ]:
#df_filtrado["pk_cid"].value_counts()

# **3. Creamos el modelo relacional de datos**


In [ ]:
#df_filtrado = customer_commercial_activity1.merge(customer_commercial_activity1, on=['pk_cid', 'pk_partition'], how='left')

In [ ]:
#customer_commercial_activity1.head()

In [ ]:
#customer_products1.head()

In [ ]:
# Crear modelo relacional de datos:   Vamos a hacer join sobre: customer_commercial_activity1, customer_products1 y customer_sociodemographics1
merged_data = customer_commercial_activity1.merge(customer_products1, on=['pk_cid', 'pk_partition'], how='left')

In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Data columns (total 21 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              object        
 1   pk_partition        datetime64[ns]
 2   entry_date          datetime64[ns]
 3   entry_channel       object        
 4   active_customer     int8          
 5   segment             object        
 6   short_term_deposit  int8          
 7   loans               int8          
 8   mortgage            int8          
 9   funds               int8          
 10  securities          int8          
 11  long_term_deposit   int8          
 12  em_account_pp       int8          
 13  credit_card         int8          
 14  payroll             int8          
 15  pension_plan        int8          
 16  payroll_account     int8          
 17  emc_account         int8          
 18  debit_card          int8          
 19  em_account_p        int8          
 20  em

In [ ]:
merged_data.isnull().sum()

pk_cid                0
pk_partition          0
entry_date            0
entry_channel         0
active_customer       0
segment               0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
em_account_pp         0
credit_card           0
payroll               0
pension_plan          0
payroll_account       0
emc_account           0
debit_card            0
em_account_p          0
em_acount             0
dtype: int64

In [ ]:
merged_data[merged_data['loans'].isnull()]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount


In [ ]:
merged_data = merged_data.merge(customer_sociodemographics1, on=['pk_cid', 'pk_partition'], how='left')


In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              object        
 1   pk_partition        datetime64[ns]
 2   entry_date          datetime64[ns]
 3   entry_channel       object        
 4   active_customer     int8          
 5   segment             object        
 6   short_term_deposit  int8          
 7   loans               int8          
 8   mortgage            int8          
 9   funds               int8          
 10  securities          int8          
 11  long_term_deposit   int8          
 12  em_account_pp       int8          
 13  credit_card         int8          
 14  payroll             int8          
 15  pension_plan        int8          
 16  payroll_account     int8          
 17  emc_account         int8          
 18  debit_card          int8          
 19  em_account_p        int8          
 20  em

In [ ]:
merged_data["pk_cid"].nunique()

456373

In [ ]:
product_description1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pk_product_ID   13 non-null     object
 1   product_desc    13 non-null     object
 2   family_product  13 non-null     object
dtypes: object(3)
memory usage: 440.0+ bytes


In [ ]:
#El siguiente merge será entre: sales1 y product_description1
merged_data_2 = sales1.merge(product_description1, left_on='product_ID', right_on='pk_product_ID', how='left')


In [ ]:
merged_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240773 entries, 0 to 240772
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   pk_sale         240773 non-null  int64         
 1   cid             240773 non-null  object        
 2   month_sale      240773 non-null  datetime64[ns]
 3   product_ID      240773 non-null  object        
 4   net_margin      240773 non-null  float64       
 5   pk_product_ID   240773 non-null  object        
 6   product_desc    240773 non-null  object        
 7   family_product  240773 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 16.5+ MB


In [ ]:
merged_data_2.isnull().sum()

pk_sale           0
cid               0
month_sale        0
product_ID        0
net_margin        0
pk_product_ID     0
product_desc      0
family_product    0
dtype: int64

In [ ]:
del(merged_data_2['product_ID'])

In [ ]:
# merged_data = merged_data.merge(merged_data_2, left_on=['pk_cid'], right_on=['cid'], how='inner')

In [ ]:
#merged_data = merged_data_2.merge(merged_data, left_on=['cid'], right_on=['pk_cid'], how='inner') # este es el merge correcto para el powerbi

In [ ]:
merged_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240773 entries, 0 to 240772
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   pk_sale         240773 non-null  int64         
 1   cid             240773 non-null  object        
 2   month_sale      240773 non-null  datetime64[ns]
 3   net_margin      240773 non-null  float64       
 4   pk_product_ID   240773 non-null  object        
 5   product_desc    240773 non-null  object        
 6   family_product  240773 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 14.7+ MB


In [ ]:
merged_data.head(3).T

,0,1,2
pk_cid,1375586,1050611,1050612
pk_partition,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00
entry_date,2018-01-01 00:00:00,2015-08-01 00:00:00,2015-08-01 00:00:00
entry_channel,KHL,KHE,KHE
active_customer,1,0,0
segment,02 - PARTICULARES,03 - UNIVERSITARIO,03 - UNIVERSITARIO
short_term_deposit,0,0,0
loans,0,0,0
mortgage,0,0,0
funds,0,0,0


In [ ]:
merged_data_2.head(3).T

,0,1,2
pk_sale,6666,6667,6668
cid,33620,35063,37299
month_sale,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-02-01 00:00:00
net_margin,952.9,1625.2,1279.7
pk_product_ID,2335,2335,2335
product_desc,short_term_deposit,short_term_deposit,short_term_deposit
family_product,investment,investment,investment


In [ ]:
merged_data_2[merged_data_2["net_margin"]==0]

,pk_sale,cid,month_sale,net_margin,pk_product_ID,product_desc,family_product


# **4. Creamos nuevas variables**

In [ ]:
#Creamos la variable, suma de productos para saber cuantos productos tiene cada cliente.

In [ ]:
columnas_productos=['short_term_deposit',
 'loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount']

In [ ]:
merged_data["Suma_productos_activos"]=merged_data[columnas_productos].sum(axis=1)

In [ ]:
merged_data.tail(20)

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,Suma_productos_activos
5962904,1166780,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,M,28,N,58274.0,1
5962905,1166779,2019-05-01,2016-08-01,KHE,1,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,H,23,N,57303.0,1
5962906,1166778,2019-05-01,2016-08-01,KHE,1,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,H,23,N,96796.0,1
5962907,1166777,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,H,23,N,157610.0,1
5962908,1166776,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,M,23,N,96796.0,1
5962909,1166775,2019-05-01,2016-08-01,KHE,1,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,H,23,N,77785.0,1
5962910,1166774,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,M,22,N,160866.0,1
5962911,1166773,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,ES,44.0,M,23,N,83822.0,0
5962912,1166772,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,H,23,N,100154.0,1
5962913,1166771,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,1,ES,50.0,M,23,N,96796.0,1


In [ ]:
merged_data["Suma_productos_activos"].value_counts()

1    3995714
0    1121507
2     528593
3     150269
4     105720
5      42890
6      14809
7       2799
8        573
9         50
Name: Suma_productos_activos, dtype: int64

revisamos que 1121507 no tienen ningún producto activo

In [ ]:
merged_data[merged_data["Suma_productos_activos"]==0]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,Suma_productos_activos
43,1050680,2018-01-01,2015-08-01,KFC,0,02 - PARTICULARES,0,0,0,0,...,0,0,0,ES,8.0,H,53,N,133092.0,0
46,1050693,2018-01-01,2015-08-01,KFC,0,02 - PARTICULARES,0,0,0,0,...,0,0,0,ES,8.0,H,37,N,104035.0,0
388,1050141,2018-01-01,2015-08-01,KHD,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,ES,15.0,H,22,N,52521.0,0
532,1050389,2018-01-01,2015-08-01,KHE,0,02 - PARTICULARES,0,0,0,0,...,0,0,0,ES,15.0,H,35,N,160539.0,0
543,1050337,2018-01-01,2015-08-01,KHE,1,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,ES,45.0,M,31,N,24625.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962890,1166735,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,ES,41.0,M,22,N,120470.0,0
5962892,1166733,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,ES,4.0,H,22,N,128305.0,0
5962893,1166744,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,ES,8.0,M,22,N,128384.0,0
5962896,1166762,2019-05-01,2016-08-01,KHE,0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,ES,46.0,H,22,N,99713.0,0


In [ ]:
# Días como cliente

merged_data["d_cliente"] = (merged_data['pk_partition'] - merged_data['entry_date']).dt.days

In [ ]:
merged_data.head(3).T

,0,1,2
pk_cid,1375586,1050611,1050612
pk_partition,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00
entry_date,2018-01-01 00:00:00,2015-08-01 00:00:00,2015-08-01 00:00:00
entry_channel,KHL,KHE,KHE
active_customer,1,0,0
segment,02 - PARTICULARES,03 - UNIVERSITARIO,03 - UNIVERSITARIO
short_term_deposit,0,0,0
loans,0,0,0
mortgage,0,0,0
funds,0,0,0


In [ ]:
# Ahora la idea es conocer cuales son los productos con los que los clientes empiezan a trabajar con Easy Bank
#lista_prod_venta1 = merged_data.groupby('product_desc')['días_para_compra'].mean()
#lista_prod_venta1.sort_values(ascending=True)

#esto lo agrego al DDSS??

In [ ]:
#Analizamos el net_margin de cada producto

resumen_net_margin = merged_data_2.groupby(['month_sale','product_desc'])['net_margin'].agg(
    mean_net_margin= 'mean',
    max_net_margin= 'max',
    min_net_margin= 'min',
    median_net_margin= 'median'
)
resumen_net_margin

mean_net_margin  max_net_margin  min_net_margin  \
month_sale product_desc                                                         
2018-02-01 credit_card              59.811961            82.3            46.1   
           debit_card               60.027940            79.0            42.9   
           em_acount                70.002502           103.9            31.4   
           emc_account              70.057249           108.7            39.7   
           funds                  1522.801031          2967.8           713.9   
...                                       ...             ...             ...   
2019-05-01 long_term_deposit      1992.950000          2271.6          1714.3   
           payroll                  69.576027           100.4            27.7   
           payroll_account          69.002133            96.6            33.9   
           pension_plan           6071.558669         17133.2          1793.2   
           securities             1375.783333          2107.4           844.0   

                              median_net_margin  
month_sale product_desc                          
2018-02-01 credit_card                    59.85  
           debit_card                     60.00  
           em_acount                      70.10  
           emc_account                    69.70  
           funds                        1509.00  
...                                         ...  
2019-05-01 long_term_deposit            1992.95  
           payroll                        69.75  
           payroll_account                68.90  
           pension_plan                 5689.10  
           securities                   1256.35  

[190 rows x 4 columns]

In [ ]:
#cuantos clientes tienen em_account_pp y em_account_p

In [ ]:
merged_data['em_account_pp'].sum()

0

In [ ]:
merged_data['em_account_p'].sum()

34

Decidimos borrar ambas condiciones también para estar alineados con product_desc

In [ ]:
del(merged_data['em_account_pp'])

In [ ]:
del(merged_data['em_account_p'])

In [ ]:
#altas de cada producto

In [ ]:
TOTAL_VENTAS = merged_data_2.pivot_table(index='month_sale', columns='product_desc', aggfunc='size', fill_value=0)
TOTAL_VENTAS

product_desc,credit_card,debit_card,em_acount,emc_account,funds,loans,long_term_deposit,mortgage,payroll,payroll_account,pension_plan,securities,short_term_deposit
month_sale,,,,,,,,,,,,,
2018-02-01,510,3708,4156,807,97,0,225,0,1966,974,2006,68,665
2018-03-01,551,2691,3521,786,118,4,249,2,1080,888,1088,39,651
2018-04-01,400,2122,3004,705,124,3,230,1,987,909,1025,33,419
2018-05-01,373,1804,3065,712,81,3,340,1,918,916,927,38,380
2018-06-01,406,1834,4711,561,51,0,371,1,1091,945,1113,34,440
2018-07-01,390,1942,12439,403,52,1,417,1,1550,1295,1621,41,450
2018-08-01,331,1943,14129,339,58,2,312,0,1073,1267,1097,73,430
2018-09-01,281,2514,17513,474,34,2,450,0,904,815,940,65,496
2018-10-01,287,3426,18988,447,37,2,453,0,1184,1077,1208,173,411


In [ ]:
TOTAL_VENTAS.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16 entries, 2018-02-01 to 2019-05-01
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   credit_card         16 non-null     int64
 1   debit_card          16 non-null     int64
 2   em_acount           16 non-null     int64
 3   emc_account         16 non-null     int64
 4   funds               16 non-null     int64
 5   loans               16 non-null     int64
 6   long_term_deposit   16 non-null     int64
 7   mortgage            16 non-null     int64
 8   payroll             16 non-null     int64
 9   payroll_account     16 non-null     int64
 10  pension_plan        16 non-null     int64
 11  securities          16 non-null     int64
 12  short_term_deposit  16 non-null     int64
dtypes: int64(13)
memory usage: 1.8 KB


In [ ]:
TOTAL_VENTAS = TOTAL_VENTAS.rename(columns={
    'credit_card': 'credit_card_venta',
    'debit_card': 'debit_card_venta',
    'em_acount': 'em_acount_venta',
    'emc_account': 'emc_account_venta',
    'funds': 'funds_venta',
    'loans': 'loans_venta',
    'long_term_deposit': 'long_term_deposit_venta',
    'mortgage': 'mortgage_venta',
    'payroll': 'payroll_venta',
    'payroll_account': 'payroll_account_venta',
    'pension_plan': 'pension_plan_venta',
    'securities': 'securities_venta',
    'short_term_deposit': 'short_term_deposit_venta'
})

In [ ]:
TOTAL_VENTAS = TOTAL_VENTAS.rename_axis('pk_partition')

In [ ]:
TOTAL_VENTAS.head(10)

product_desc,credit_card_venta,debit_card_venta,em_acount_venta,emc_account_venta,funds_venta,loans_venta,long_term_deposit_venta,mortgage_venta,payroll_venta,payroll_account_venta,pension_plan_venta,securities_venta,short_term_deposit_venta
pk_partition,,,,,,,,,,,,,
2018-02-01,510,3708,4156,807,97,0,225,0,1966,974,2006,68,665
2018-03-01,551,2691,3521,786,118,4,249,2,1080,888,1088,39,651
2018-04-01,400,2122,3004,705,124,3,230,1,987,909,1025,33,419
2018-05-01,373,1804,3065,712,81,3,340,1,918,916,927,38,380
2018-06-01,406,1834,4711,561,51,0,371,1,1091,945,1113,34,440
2018-07-01,390,1942,12439,403,52,1,417,1,1550,1295,1621,41,450
2018-08-01,331,1943,14129,339,58,2,312,0,1073,1267,1097,73,430
2018-09-01,281,2514,17513,474,34,2,450,0,904,815,940,65,496
2018-10-01,287,3426,18988,447,37,2,453,0,1184,1077,1208,173,411


In [ ]:
TOTAL_STOCK = merged_data.groupby(['pk_partition']).agg(
    {
        'credit_card':np.sum,
        'debit_card':np.sum,
        'em_acount':np.sum,
        'emc_account':np.sum,
        'funds':np.sum,
        'loans':np.sum,
        'long_term_deposit':np.sum,
        'mortgage':np.sum,
        'payroll':np.sum,
        'payroll_account':np.sum,
        'pension_plan':np.sum,
        'securities':np.sum,
        'short_term_deposit':np.sum
    }
)
TOTAL_STOCK




,credit_card,debit_card,em_acount,emc_account,funds,loans,long_term_deposit,mortgage,payroll,payroll_account,pension_plan,securities,short_term_deposit
pk_partition,,,,,,,,,,,,,
2018-01-01,3325,24696,215293,15320,786,19,4884,15,8145,13478,8835,932,883
2018-02-01,3282,25448,217098,15827,872,19,4956,15,8889,14246,9496,965,1343
2018-03-01,3511,27046,218683,16428,969,23,4965,17,9735,14989,9882,965,1664
2018-04-01,3725,27764,219816,16947,1084,24,5107,17,9905,15825,10560,983,1740
2018-05-01,3797,27911,221291,17569,1145,27,5356,18,9994,16697,10148,1012,1455
2018-06-01,4002,28828,224328,17950,1159,27,5588,19,10946,15540,11635,1019,1244
2018-07-01,4137,29578,234324,18185,1172,28,5796,20,11888,16916,12779,1053,1282
2018-08-01,4206,29205,245980,18333,1219,29,5923,20,11383,18118,12126,1133,1335
2018-09-01,4289,31684,261210,18618,1237,31,6158,20,11850,18862,12624,1201,1386


In [ ]:
shifted_columns = []
for column in TOTAL_STOCK.columns:
    new_column_name = column + '_shifted'
    shifted_column = TOTAL_STOCK[column].shift(1)
    TOTAL_STOCK[new_column_name] = shifted_column
    shifted_columns.append(new_column_name)


In [ ]:
INVENTARIO = TOTAL_STOCK.merge(TOTAL_VENTAS, on=['pk_partition'], how='left')

In [ ]:
INVENTARIO.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17 entries, 2018-01-01 to 2019-05-01
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   credit_card                 17 non-null     int64  
 1   debit_card                  17 non-null     int64  
 2   em_acount                   17 non-null     int64  
 3   emc_account                 17 non-null     int64  
 4   funds                       17 non-null     int64  
 5   loans                       17 non-null     int8   
 6   long_term_deposit           17 non-null     int64  
 7   mortgage                    17 non-null     int8   
 8   payroll                     17 non-null     int64  
 9   payroll_account             17 non-null     int64  
 10  pension_plan                17 non-null     int64  
 11  securities                  17 non-null     int64  
 12  short_term_deposit          17 non-null     int64  
 13  credit_card_shift

In [ ]:
productos = ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'credit_card', 'payroll', 'pension_plan', 'payroll_account', 'emc_account', 'debit_card', 'em_acount']

for producto in productos:
    column_shifted = producto + '_shifted'
    column_venta = producto + '_venta'
    column_baja = producto + '_baja'


    def calculate_total(row, column_shifted, producto, column_venta):
      result = row[column_shifted] - row[producto] + row[column_venta]
      if result < 0:
        return 0
      else:
        return result

    INVENTARIO[column_baja] = INVENTARIO.apply(calculate_total, axis=1, args=(column_shifted, producto, column_venta))


In [ ]:
INVENTARIO.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17 entries, 2018-01-01 to 2019-05-01
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   credit_card                 17 non-null     int64  
 1   debit_card                  17 non-null     int64  
 2   em_acount                   17 non-null     int64  
 3   emc_account                 17 non-null     int64  
 4   funds                       17 non-null     int64  
 5   loans                       17 non-null     int8   
 6   long_term_deposit           17 non-null     int64  
 7   mortgage                    17 non-null     int8   
 8   payroll                     17 non-null     int64  
 9   payroll_account             17 non-null     int64  
 10  pension_plan                17 non-null     int64  
 11  securities                  17 non-null     int64  
 12  short_term_deposit          17 non-null     int64  
 13  credit_card_shift

In [ ]:
INVENTARIO = INVENTARIO.T

In [ ]:
INVENTARIO

pk_partition,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01
credit_card,3325.0,3282.0,3511.0,3725.0,3797.0,4002.0,4137.0,4206.0,4289.0,4385.0,4512.0,4536.0,4493.0,4542.0,4576.0,4747.0,4801.0
debit_card,24696.0,25448.0,27046.0,27764.0,27911.0,28828.0,29578.0,29205.0,31684.0,34372.0,35466.0,37430.0,37350.0,39399.0,41390.0,42140.0,43261.0
em_acount,215293.0,217098.0,218683.0,219816.0,221291.0,224328.0,234324.0,245980.0,261210.0,277707.0,284900.0,288928.0,291786.0,292988.0,295046.0,295844.0,296380.0
emc_account,15320.0,15827.0,16428.0,16947.0,17569.0,17950.0,18185.0,18333.0,18618.0,18844.0,19494.0,20043.0,20921.0,21796.0,22480.0,23455.0,24751.0
funds,786.0,872.0,969.0,1084.0,1145.0,1159.0,1172.0,1219.0,1237.0,1249.0,1282.0,1322.0,1330.0,1316.0,1322.0,1320.0,1315.0
loans,19.0,19.0,23.0,24.0,27.0,27.0,28.0,29.0,31.0,33.0,32.0,30.0,28.0,29.0,29.0,30.0,30.0
long_term_deposit,4884.0,4956.0,4965.0,5107.0,5356.0,5588.0,5796.0,5923.0,6158.0,6380.0,6447.0,6749.0,6659.0,6658.0,6537.0,6368.0,6129.0
mortgage,15.0,15.0,17.0,17.0,18.0,19.0,20.0,20.0,20.0,19.0,19.0,19.0,20.0,20.0,20.0,23.0,23.0
payroll,8145.0,8889.0,9735.0,9905.0,9994.0,10946.0,11888.0,11383.0,11850.0,12454.0,13032.0,14452.0,12058.0,14378.0,15114.0,15231.0,16333.0
payroll_account,13478.0,14246.0,14989.0,15825.0,16697.0,15540.0,16916.0,18118.0,18862.0,19945.0,21437.0,21359.0,22324.0,23264.0,24390.0,25501.0,26529.0


In [ ]:
INVENTARIO.columns.tolist()

[Timestamp('2018-01-01 00:00:00'),
 Timestamp('2018-02-01 00:00:00'),
 Timestamp('2018-03-01 00:00:00'),
 Timestamp('2018-04-01 00:00:00'),
 Timestamp('2018-05-01 00:00:00'),
 Timestamp('2018-06-01 00:00:00'),
 Timestamp('2018-07-01 00:00:00'),
 Timestamp('2018-08-01 00:00:00'),
 Timestamp('2018-09-01 00:00:00'),
 Timestamp('2018-10-01 00:00:00'),
 Timestamp('2018-11-01 00:00:00'),
 Timestamp('2018-12-01 00:00:00'),
 Timestamp('2019-01-01 00:00:00'),
 Timestamp('2019-02-01 00:00:00'),
 Timestamp('2019-03-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-05-01 00:00:00')]

# **5. Exportamos los dataset**

In [ ]:
# Exportar datos procesados en formato .csv para su uso en Power BI
merged_data.to_csv("informacion_clientes.csv", index=False)
INVENTARIO.to_csv("informacion_productos.csv", index=True)
merged_data_2.to_csv("informacion_ventas.csv", index=False)

In [ ]:
# Lo descargamos
files.download("informacion_clientes.csv")
files.download("informacion_productos.csv")
files.download("informacion_ventas.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>